In [4]:
import pathlib
import geopandas
import pandas
import numpy

## Computer specific file paths - make sure you change these

In [ ]:
cache_path = pathlib.Path(r"C:\Users\pearsonra\Documents\data\Westport\caches\NZ20_Westport\large_full_river")
flow_path = pathlib.Path(r"C:\Users\pearsonra\Documents\data\river_bathemetry\mannings_n")

## Standard file names generated by geofabrics

In [16]:
river_characteristics = geopandas.read_file(cache_path / "river_characteristics.geojson")
channel = geopandas.read_file(cache_path / "rec_channel_6307000000.geojson")
flow_and_friction = pandas.read_csv(flow_path / "bathy.csv.gz")

In [25]:
flow_name = "flow"
friction_name = "mannings_n"

slope_name = "slope_mean_2.0km"
min_z_name = "min_z_centre_unimodal"
width_name = "widths_mean_0.25km"
flat_width_name = "flat_widths_mean_0.25km"
threshold_name = "thresholds_mean_0.25km"

## Match the river characteristics to the REC flow and frictions

In [17]:
# Match each channel midpoint to a nzsegment ID - based on what channel reach is closest
river_characteristics["nzsegment"] = (
    numpy.ones(len(river_characteristics["widths"]), dtype=float) * numpy.nan
)
for i, row in river_characteristics.iterrows():
    if row.geometry is not None and not row.geometry.is_empty:
        distances = channel.distance(river_characteristics.loc[i].geometry)
        river_characteristics.loc[i, ("nzsegment")] = channel[
            distances == distances.min()
        ]["nzsegment"].min()
# Fill in any missing values
river_characteristics["nzsegment"] = (
    river_characteristics["nzsegment"].fillna(method="ffill").fillna(method="bfill")
)
river_characteristics["nzsegment"] = river_characteristics["nzsegment"].astype("int")

# Add the friction and flow values to the widths and slopes
river_characteristics[friction_name] = numpy.zeros(len(river_characteristics["nzsegment"]), dtype=int)
river_characteristics[flow_name] = numpy.zeros(len(river_characteristics["nzsegment"]), dtype=int)

for nzsegment in river_characteristics["nzsegment"].unique():
    river_characteristics.loc[
        river_characteristics["nzsegment"] == nzsegment, (friction_name)
    ] = flow_and_friction[flow_and_friction["nzsegment"] == nzsegment]["n"].unique()[0]
    river_characteristics.loc[river_characteristics["nzsegment"] == nzsegment, (flow_name)] = flow_and_friction[
        flow_and_friction["nzsegment"] == nzsegment
    ][flow_name].unique()[0]

## Caclulate the depths

In [29]:
def calculate_depth(alpha, beta, K_0, slope, width, flow):
    full_bank_depth = ( flow / (K_0 * width * slope ** beta) ) ** (1 / (1 + alpha))
    return full_bank_depth

In [30]:
def calculate_neal_et_al(slope, width, flow, n):
    alpha = 0.6
    beta = 0.5
    K_0 = 1/n
    return calculate_depth(alpha, beta, K_0, slope, width, flow)

In [31]:
def calculate_rupp_and_smart(slope, width, flow):
    alpha = 0.745
    beta = 0.305
    K_0 = 6.16
    return calculate_depth(alpha, beta, K_0, slope, width, flow)

In [33]:
index=0; row = river_characteristics.iloc[index]
depth_neal = calculate_neal_et_al(row[slope_name], row[width_name], row[flow_name], row[friction_name])
depth_rupp = calculate_rupp_and_smart(row[slope_name], row[width_name], row[flow_name])
print(f"We get depths of {depth_neal} for Neal et al, and {depth_rupp} for Rupp et al")

We get depths of 13.929210707748444 for Neal et al, and 8.61702242052938 for Rupp et al
